In [66]:
import networkx as nx
from networkx.algorithms.community import greedy_modularity_communities
from networkx.algorithms.community import modularity
import matplotlib.pyplot as plt
from collections import Counter
import pandas as pd
import os


In [71]:


directory = '/Users/sunechristiansen/sune/network_analysis/project/parasites_network_analysis/projections_with_backbonings/projection_parasites/resource_allocation_naive.csv'

In [80]:
df = pd.read_csv(directory)
df.rename(columns={'src': 'source', 'trg': 'target', 'nij': 'weight'}, inplace=True)
df.drop(columns=['Unnamed: 0'], inplace=True)

In [82]:
# Create the graph with weights
G = nx.from_pandas_edgelist(df, source='source', target='target', edge_attr='weight')

In [112]:
atribute_df = pd.read_csv('/Users/sunechristiansen/sune/network_analysis/project/parasites_network_analysis/data/nodes.csv')

extra_attributes = pd.read_csv('/Users/sunechristiansen/Downloads/helminths (1).csv', encoding='ISO-8859-1')

In [113]:
extra_attributes

,Unnamed: 0,Host,Parasite,ParasiteFull,group,locality,hostgroup
0,2,Squalus blainville,Acanthocephaloides incrassatus,"Acanthocephaloides incrassatus (Molin, 1858)",Acanthocephala,Adriatic Sea,Chondrichthyes
1,3,Gobius bucchichi,Acanthocephaloides propinquus,"Acanthocephaloides propinquus (Dujardin, 1845)",Acanthocephala,Adriatic Sea,Teleostei
2,4,Gobius cruentatus,Acanthocephaloides propinquus,"Acanthocephaloides propinquus (Dujardin, 1845)",Acanthocephala,Adriatic Sea,Teleostei
3,5,Gobius niger,Acanthocephaloides propinquus,"Acanthocephaloides propinquus (Dujardin, 1845)",Acanthocephala,Adriatic Sea,Teleostei
4,6,Lesueurigobius friesii,Acanthocephaloides propinquus,"Acanthocephaloides propinquus (Dujardin, 1845)",Acanthocephala,Adriatic Sea,Teleostei
...,...,...,...,...,...,...,...
146678,206153,Bulinus globosus,Schistosoma mattheei,"Schistosoma mattheei Veglia & Le Roux, 1929",Trematoda,Zimbabwe,NaN
146679,206154,Giraffa cameleopardus,Schistosoma mattheei,"Schistosoma mattheei Veglia & Le Roux, 1929",Trematoda,Zimbabwe,NaN
146680,206155,Homo sapiens,Schistosoma mattheei,"Schistosoma mattheei Veglia & Le Roux, 1929",Trematoda,Zimbabwe,Mammalia
146681,206156,Kobus ellipsiprymnus,Schistosoma mattheei,"Schistosoma mattheei Veglia & Le Roux, 1929",Trematoda,Zimbabwe,Mammalia


In [114]:
is_parasites = atribute_df[atribute_df[' is_host'] == 0]
is_parasites

,# index,name,full_name,locality,group,is_host,_pos
1,1,Acanthocephaloides incrassatus,NaN,NaN,Teleostei,0,"array([39.44431659, 25.20409242])"
3,3,Acanthocephaloides propinquus,NaN,NaN,Teleostei,0,"array([37.69593368, 27.68332545])"
9,9,Acanthogyrus lizae,NaN,NaN,Maxillopoda,0,"array([56.32348974, 4.78947382])"
11,11,Neoechinorhynchus agilis,NaN,NaN,Teleostei,0,"array([52.25395966, 5.96265359])"
14,14,Rhadinorhynchus pristis,NaN,NaN,Teleostei,0,"array([50.7988151 , 49.71082799])"
...,...,...,...,...,...,...,...
30508,30508,Tamerlania meruli,NaN,NaN,Aves,0,"array([-34.9902523 , -36.20314686])"
30509,30509,Tanaisia elliptica,NaN,NaN,NaN,0,"array([-19.07101244, -48.59314897])"
30513,30513,Cynodiplostomum namrui,NaN,NaN,NaN,0,"array([ 70.69624512, -66.57151265])"
30514,30514,Fasciola nyanzae,NaN,NaN,Mammalia,0,"array([ 78.34662372, -96.61181142])"


In [118]:
# Merge the DataFrames based on the conditions
merged_df = is_parasites.merge(
    extra_attributes,
    left_on=[' name', ' group'],  # Columns in attribute_df
    right_on=['Parasite', 'hostgroup'],  # Columns in extra_attributes
    how='inner'  # Use 'inner' to keep only matching rows
)

# Display the merged DataFrame
print(merged_df)


        # index                            name  full_name  locality  \
0             1  Acanthocephaloides incrassatus        NaN       NaN   
1             1  Acanthocephaloides incrassatus        NaN       NaN   
2             1  Acanthocephaloides incrassatus        NaN       NaN   
3             1  Acanthocephaloides incrassatus        NaN       NaN   
4             1  Acanthocephaloides incrassatus        NaN       NaN   
...         ...                             ...        ...       ...   
106013    30508               Tamerlania meruli        NaN       NaN   
106014    30509              Tanaisia elliptica        NaN       NaN   
106015    30513          Cynodiplostomum namrui        NaN       NaN   
106016    30514                Fasciola nyanzae        NaN       NaN   
106017    30515         Prudhoella rhodesiensis        NaN       NaN   

            group   is_host                                 _pos  Unnamed: 0  \
0       Teleostei         0    array([39.44431659, 25.2

In [125]:
groupe_merge = merged_df.groupby('# index')

In [143]:
merged_df.columns


Index(['# index', ' name', ' full_name', ' locality', ' group', ' is_host',
       ' _pos', 'Unnamed: 0', 'Host', 'Parasite', 'ParasiteFull', 'group',
       'locality', 'hostgroup'],
      dtype='object')

In [161]:

# Create a dictionary of the attributes of list of each group locality and host
dict_attributes = {}
for group, data in groupe_merge:
    print(group)
    dict_attributes[group] = [data['Host'].tolist(), data['locality'].tolist()]


1
3
9
11
14
16
18
20
39
42
45
48
50
52
54
58
63
66
67
69
72
76
87
88
89
90
92
97
98
101
103
110
119
126
129
130
131
133
135
149
155
187
188
202
203
204
209
210
211
212
213
214
217
218
220
221
224
225
226
227
229
231
233
235
237
240
242
244
246
248
250
252
254
255
256
258
260
262
264
266
270
271
272
274
276
277
279
281
283
285
286
289
291
293
295
296
298
300
302
305
307
309
311
320
321
324
326
328
330
332
334
335
337
339
341
342
344
354
363
364
382
383
384
386
388
390
400
402
405
407
409
413
416
421
424
427
429
431
433
435
437
438
439
440
441
442
444
446
448
450
453
454
456
458
461
463
467
484
500
501
503
510
512
514
515
517
522
525
531
533
559
560
563
579
581
583
587
612
614
636
638
643
644
645
647
652
709
710
711
712
715
717
721
723
725
727
729
730
733
738
740
741
746
751
755
757
759
762
766
768
769
770
772
775
777
779
782
786
787
788
790
792
795
797
802
805
807
809
810
815
818
821
823
825
829
831
854
855
864
878
880
882
891
894
898
900
902
904
912
920
923
932
936
938
940
942
945
946


In [167]:
# add the attributes to is_parasites from the dictionary keys == # index
# Map the dictionary values to new columns in is_parasites
is_parasites['Host_name'] = is_parasites['# index'].map(lambda x: dict_attributes[x][0] if x in dict_attributes else None)
is_parasites['locality_host'] = is_parasites['# index'].map(lambda x: dict_attributes[x][1] if x in dict_attributes else None)

# Display the updated DataFrame
print(is_parasites.head())


    # index                            name  full_name  locality        group  \
1         1  Acanthocephaloides incrassatus        NaN       NaN    Teleostei   
3         3   Acanthocephaloides propinquus        NaN       NaN    Teleostei   
9         9              Acanthogyrus lizae        NaN       NaN  Maxillopoda   
11       11        Neoechinorhynchus agilis        NaN       NaN    Teleostei   
14       14         Rhadinorhynchus pristis        NaN       NaN    Teleostei   

     is_host                               _pos Host  \
1          0  array([39.44431659, 25.20409242])  nan   
3          0  array([37.69593368, 27.68332545])  nan   
9          0  array([56.32348974,  4.78947382])  nan   
11         0  array([52.25395966,  5.96265359])  nan   
14         0  array([50.7988151 , 49.71082799])  nan   

                                            Host_name  \
1   [Symphodus ocellatus, Symphodus tinca, Gobius ...   
3   [Gobius bucchichi, Gobius cruentatus, Gobius n...   
9   [

/var/folders/vw/nff8_41x38jcbpv0nvrcxv700000gn/T/ipykernel_99882/3802806689.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  is_parasites['Host_name'] = is_parasites['# index'].map(lambda x: dict_attributes[x][0] if x in dict_attributes else None)
/var/folders/vw/nff8_41x38jcbpv0nvrcxv700000gn/T/ipykernel_99882/3802806689.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  is_parasites['locality_host'] = is_parasites['# index'].map(lambda x: dict_attributes[x][1] if x in dict_attributes else None)


In [172]:
is_parasites.rename(columns={' # index': 'node_id', ' name': 'parasite_name', ' group': 'parasite_group'}, inplace=True)

/var/folders/vw/nff8_41x38jcbpv0nvrcxv700000gn/T/ipykernel_99882/494605015.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  is_parasites.rename(columns={' # index': 'node_id', ' name': 'parasite_name', ' group': 'parasite_group'}, inplace=True)


In [174]:
is_parasites.drop(columns=[' full_name', ' locality', 'Host'], inplace=True)

/var/folders/vw/nff8_41x38jcbpv0nvrcxv700000gn/T/ipykernel_99882/998199618.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  is_parasites.drop(columns=[' full_name', ' locality', 'Host'], inplace=True)


In [176]:
is_parasites.drop(columns=[' _pos'], inplace=True)

/var/folders/vw/nff8_41x38jcbpv0nvrcxv700000gn/T/ipykernel_99882/2530705198.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  is_parasites.drop(columns=[' _pos'], inplace=True)


In [182]:
is_parasites

,node_id,parasite_name,parasite_group,is_host,Host_name,locality_host
1,1,Acanthocephaloides incrassatus,Teleostei,0,"[Symphodus ocellatus, Symphodus tinca, Gobius ...","[Atlantic Ocean, Atlantic Ocean, Atlantic Ocea..."
3,3,Acanthocephaloides propinquus,Teleostei,0,"[Gobius bucchichi, Gobius cruentatus, Gobius n...","[Adriatic Sea, Adriatic Sea, Adriatic Sea, Adr..."
9,9,Acanthogyrus lizae,Maxillopoda,0,"[Halectinosoma herdmani, Halectinosoma herdman...","[Atlantic Ocean, English Channel, Northeast At..."
11,11,Neoechinorhynchus agilis,Teleostei,0,"[Chelon labrosus, Liza aurata, Chelon labrosus...","[Adriatic Sea, Adriatic Sea, Atlantic Ocean, A..."
14,14,Rhadinorhynchus pristis,Teleostei,0,"[Pseudorhombus arsius, Alectis alexandrinus, A...","[Arabian Sea, Atlantic Ocean, Atlantic Ocean, ..."
...,...,...,...,...,...,...
30508,30508,Tamerlania meruli,Aves,0,[Turdus merula],[Yugoslavia]
30509,30509,Tanaisia elliptica,NaN,0,[Hydrochelidon nigra],[Yugoslavia]
30513,30513,Cynodiplostomum namrui,NaN,0,[Hydrictis maculicollis],[Zimbabwe]
30514,30514,Fasciola nyanzae,Mammalia,0,[Hippopotamus amphibius],[Zimbabwe]


In [183]:
is_parasites.to_csv('/Users/sunechristiansen/sune/network_analysis/project/parasites_network_analysis/data/parasites.csv', index=False)

In [ ]:

for i in range(len(files)):
    # Read the CSV file
    df = pd.read_csv('/Users/sunechristiansen/sune/network_analysis/project/parasites_network_analysis/projections_with_backbonings/' + files[i])

    # Rename columns
    df.rename(columns={'src': 'source', 'trg': 'target', 'nij': 'weight'}, inplace=True)

    # Create the graph with weights
    G = nx.from_pandas_edgelist(df, source='source', target='target', edge_attr='weight')

    print(f'name: {files[i]}, edges: {len(G.edges(data=True))}, nodes: {len(G.nodes)}')


name: simple_disparity_filter.csv, edges: 598944, nodes: 17426
name: pearson_noise_corrected.csv, edges: 109730, nodes: 15369
name: hyperbolic_high_salience_skeleton.csv, edges: 598944, nodes: 17426
name: resource_allocation_naive.csv, edges: 598944, nodes: 17426
name: simple_noise_corrected.csv, edges: 598944, nodes: 17426


In [2]:
resource_all_df = pd.read_csv('resource_allocation_naive.csv')
# Rename columns
resource_all_df.rename(columns={'src': 'source', 'trg': 'target', 'nij': 'weight'}, inplace=True)

# Create the graph with weights
G = nx.from_pandas_edgelist(resource_all_df, source='source', target='target', edge_attr='weight')

In [ ]:


""" # Find all connected components
connected_components = list(nx.connected_components(filtered_G))

# Find the largest connected component
largest_component = max(connected_components, key=len)

# Create a subgraph of the largest connected component
largest_subgraph = filtered_G.subgraph(largest_component) """

# Print details
print(f"Number of nodes in largest component: {len(largest_subgraph.nodes)}")
print(f"Number of edges in largest component: {len(largest_subgraph.edges)}")


Number of nodes in largest component: 15899
Number of edges in largest component: 438649


In [3]:
# Brute force find highest modularity resource allocation plus threshold:
for i in range(100):
    threshold = i/100
    # Get all edge weights
    edge_weights = nx.get_edge_attributes(G, 'weight')

    # Filter edges with weights above the threshold
    filtered_edges = [(u, v, w) for (u, v), w in edge_weights.items() if w >= threshold]

    # Create a new graph with the filtered edges
    filtered_G = nx.Graph()  # Use nx.DiGraph() if the original graph is directed
    filtered_G.add_weighted_edges_from(filtered_edges)

    communities = greedy_modularity_communities(filtered_G, weight=None)

    modularity_r = modularity(filtered_G, communities)
    print("threshold: ", threshold, " & modularity: ", modularity_r)
    

KeyboardInterrupt: 